In [ ]:
"""
Train one DistilBERT model per classification task (queue, type, priority)
and persist each model (weights and tokenizer) to disk.
"""

# --------------------------------------------------------------------
# Imports
# --------------------------------------------------------------------
import os
import random
from collections import Counter

import numpy as np
import pandas as pd
import torch
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
from transformers import (
    DistilBertTokenizer,
    DistilBertForSequenceClassification,
    get_scheduler,
)

# --------------------------------------------------------------------
# Reproducibility helpers
# --------------------------------------------------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# --------------------------------------------------------------------
# Configuration
# --------------------------------------------------------------------
MODEL_NAME = "distilbert-base-uncased"
TASKS = ["queue", "type", "priority"]
MAX_LENGTH = 256
BATCH_SIZE = 16
EPOCHS = 4
LR = 2e-5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

OUT_DIR = "saved_models"
os.makedirs(OUT_DIR, exist_ok=True)

# --------------------------------------------------------------------
# Load and prepare data
# --------------------------------------------------------------------
df = pd.read_csv("/content/dataset-tickets-multi-lang-4-20k.csv")
df["text"] = df["subject"].fillna("") + " " + df["body"].fillna("")

encoders = {
    col: {v: k for k, v in enumerate(df[col].unique())} for col in TASKS
}
decoders = {col: {v: k for k, v in encoders[col].items()} for col in TASKS}

for col in TASKS:
    df[col] = df[col].map(encoders[col])

train_df, val_df = train_test_split(df, test_size=0.2, random_state=SEED)

# --------------------------------------------------------------------
# Dataset class
# --------------------------------------------------------------------
class TicketDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, label_column: str, tokenizer):
        self.texts = dataframe["text"].tolist()
        self.labels = dataframe[label_column].tolist()
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            max_length=MAX_LENGTH,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
        }

# --------------------------------------------------------------------
# Training helpers
# --------------------------------------------------------------------
def train_model(model, train_loader, val_loader):
    optimizer = AdamW(model.parameters(), lr=LR)
    total_steps = len(train_loader) * EPOCHS
    scheduler = get_scheduler(
        "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=total_steps
    )

    best_val_acc = -1.0
    best_state_dict = None

    for epoch in range(1, EPOCHS + 1):
        model.train()
        running_loss = 0.0
        for batch in tqdm(train_loader, leave=False):
            batch = {k: v.to(DEVICE) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss

            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

            running_loss += loss.item()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for batch in val_loader:
                batch = {k: v.to(DEVICE) for k, v in batch.items()}
                outputs = model(**batch)
                preds = outputs.logits.argmax(dim=1)
                correct += (preds == batch["labels"]).sum().item()
                total += batch["labels"].size(0)

        val_acc = correct / total
        print(f"  Epoch {epoch:2d} | Train loss {running_loss/len(train_loader):.4f} | Val acc {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state_dict = {k: v.cpu() for k, v in model.state_dict().items()}

    model.load_state_dict(best_state_dict)
    return model

def build_train_and_save(task_name, num_classes, tokenizer):
    print(f"\n=== Training model for task: '{task_name}' ===")
    train_ds = TicketDataset(train_df, task_name, tokenizer)
    val_ds = TicketDataset(val_df, task_name, tokenizer)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

    model = DistilBertForSequenceClassification.from_pretrained(
        MODEL_NAME, num_labels=num_classes
    ).to(DEVICE)

    model = train_model(model, train_loader, val_loader)

    save_path = os.path.join(OUT_DIR, task_name)
    model.save_pretrained(save_path)
    if not os.path.exists(os.path.join(save_path, "tokenizer_config.json")):
        tokenizer.save_pretrained(save_path)

    print(f"\n✔ Saved '{task_name}' model to {save_path}\n")
    return model

# --------------------------------------------------------------------
# Train all task-specific models
# --------------------------------------------------------------------
if __name__ == "__main__":
    tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)

    models = {
        task: build_train_and_save(task, len(encoders[task]), tokenizer) for task in TASKS
    }

    # ----------------------------------------------------------------
    # Evaluation
    # ----------------------------------------------------------------
    y_true = {t: [] for t in TASKS}
    y_pred = {t: [] for t in TASKS}

    @torch.inference_mode()
    def predict(task, text):
        model = models[task]
        encoded = tokenizer(
            text,
            max_length=MAX_LENGTH,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        ).to(DEVICE)
        logits = model(**encoded).logits.squeeze(0)
        return logits.argmax().item()

    print("\n=== Validation-set reports ===")
    for _idx, row in tqdm(val_df.iterrows(), total=len(val_df)):
        text = row["text"]
        for task in TASKS:
            y_true[task].append(row[task])
            y_pred[task].append(predict(task, text))

    for task in TASKS:
        print(f"\n[{task.upper()}]")
        print(
            classification_report(
                y_true[task],
                y_pred[task],
                target_names=[decoders[task][i] for i in range(len(encoders[task]))],
                digits=3,
                zero_division=0,
            )
        )

    # ----------------------------------------------------------------
    # Example – quick test
    # ----------------------------------------------------------------
    sample_ticket = """
    Urgent: Data breach identified in hospital network.
    Patient records system compromised. Immediate action required.
    """
    print("\nSample ticket prediction:")
    for task in TASKS:
        pred_idx = predict(task, sample_ticket)
        print(f"  {task:8s}: {decoders[task][pred_idx]}")



vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]


=== Training model for task: 'queue' ===


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

  Epoch  3 | Train loss 0.9179 | Val acc 0.5108


  0%|          | 0/1000 [00:00<?, ?it/s]

  Epoch  4 | Train loss 0.8105 | Val acc 0.5218

✔ Saved 'priority' model to saved_models/priority


=== Validation-set reports ===


  0%|          | 0/4000 [00:00<?, ?it/s]


[QUEUE]
                                 precision    recall  f1-score   support

                General Inquiry      0.000     0.000     0.000        54
               Customer Service      0.351     0.376     0.363       652
              Technical Support      0.454     0.665     0.540      1179
                     IT Support      0.379     0.201     0.262       458
                Product Support      0.345     0.352     0.348       707
           Billing and Payments      0.732     0.718     0.725       422
Service Outages and Maintenance      0.520     0.464     0.491       140
                Human Resources      0.737     0.212     0.329        66
          Returns and Exchanges      0.408     0.099     0.159       203
            Sales and Pre-Sales      1.000     0.034     0.065       119

                       accuracy                          0.444      4000
                      macro avg      0.493     0.312     0.328      4000
                   weighted avg      0.4